# PyTTI-Tools Colab Notebook

If you are using PyTTI-tools from a local jupyter server, you might have a better experience with the "_local" notebook: https://github.com/pytti-tools/pytti-notebook/blob/main/pyttitools-PYTTI_local.ipynb

If you are planning to use google colab with the "local runtime" option: this is still the notebook you want.

## A very brief history of this notebook

The tools and techniques below were pioneered in 2021 by a diverse and distributed collection of amazingly talented ML practitioners, researchers, and artists. The short version of this history is that Katherine Crowson ([@RiversHaveWings](https://twitter.com/RiversHaveWings)) published a notebook inspired by work done by [@advadnoun](https://twitter.com/advadnoun). Katherine's notebook spawned a litany of variants, each with their own twist on the technique or adding a feature to someone else's work. Henry Rachootin ([@sportsracer48](https://twitter.com/sportsracer48)) collected several of the most interesting notebooks and stuck the important bits together with bublegum and scotch tape. Thus was born PyTTI, and there was much rejoicing in sportsracer48's patreon, where it was shared in closed beta for several months. David Marx ([@DigThatData](https://twitter.com/DigThatData)) offered to help tidy up the mess, and sportsracer48 encouraged him to run wild with it. David's contributions snowballed into [PyTTI-Tools](https://github.com/pytti-tools), the engine this notebook sits on top of!

If you would like to contribute, receive support, or even just suggest an improvement to the documentation, our issue tracker can be found here: https://github.com/pytti-tools/pytti-core/issues

# Instructions

Detailed documentation can be found here: https://pytti-tools.github.io/pytti-book/intro.html

* Syntax for text prompts and scenes: https://pytti-tools.github.io/pytti-book/SceneDSL.html
* Descriptions of all settings: https://pytti-tools.github.io/pytti-book/Settings.html


#Step 1: Setup
Run the cells in this section once for each runtime, or after a factory reset.

In [2]:
# This cell should only be run once
drive_mounted = False
gdrive_fpath = '.'

In [4]:
#@title 1.1 Mount google drive (optional)
#@markdown Mounting your drive is optional but recommended. You can even restore from google randomly
#@markdown kicking you out if you mount your drive.

from pathlib import Path

mount_gdrive = False # @param{type:"boolean"}

if mount_gdrive and not drive_mounted:
  from google.colab import drive

  gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
  gdrive_subdirectory = 'MyDrive/pytti_tools' #@param{type:"string"}
  gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
  try:
    drive.mount(gdrive_mountpoint, force_remount = True)
    !mkdir -p {gdrive_fpath}
    %cd {gdrive_fpath}
    drive_mounted = True
  except OSError:
    print(
        "\n\n-----[PYTTI-TOOLS]-------\n\n"
        "If you received a scary OSError and your drive"
        " was already mounted, ignore it."
        "\n\n-----[PYTTI-TOOLS]-------\n\n"
        )
    raise

In [5]:
#@title 1.2 NVIDIA-SMI (optional)
#@markdown View information about your runtime GPU.
#@markdown Google will connect you to an industrial strength GPU, which is needed to run
#@markdown this notebook. You can also disable error checking on your GPU to get some
#@markdown more VRAM, at a marginal cost to stability. You will have to restart the runtime after
#@markdown disabling it.
enable_error_checking = False#@param {type:"boolean"}
if enable_error_checking:
  !nvidia-smi
else:
  !nvidia-smi
  !nvidia-smi -i 0 -e 0

zsh:1: command not found: nvidia-smi
zsh:1: command not found: nvidia-smi


In [6]:
#@title 1.3 Install everything else
#@markdown Run this cell on a fresh runtime to install the libraries and modules.
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}

def flush_reqs():
    !rm -r pytti-core

def install_everything():
    if path_exists('./pytti-core'):
        try:
            flush_reqs()
        except Exception as ex:
            logger.warning(
                str(ex)
            )
            logger.warning(
                "A `pytti` folder already exists and could not be deleted."
                "If you encounter problems, try deleting that folder and trying again."
                "Please report this and any other issues here: "
                "https://github.com/pytti-tools/pytti-notebook/issues/new",
                exc_info=True)

    !git clone --recurse-submodules -j8 https://github.com/wizardhead/pytti-core

    !pip install kornia pytorch-lightning transformers
    !pip install jupyter loguru einops PyGLM ftfy regex tqdm hydra-core exrex
    !pip install seaborn adjustText bunch matplotlib-label-lines
    !pip install --upgrade gdown
    
    !pip install ./pytti-core/vendor/AdaBins
    !pip install ./pytti-core/vendor/CLIP
    !pip install ./pytti-core/vendor/GMA
    !pip install ./pytti-core/vendor/taming-transformers
    !pip install ./pytti-core

    !mkdir -p images_out
    !mkdir -p videos

    from pytti.Notebook import change_tqdm_color
    change_tqdm_color()

try:
    from adjustText import adjust_text
    import pytti, torch
    everything_installed = True
except ModuleNotFoundError:
    everything_installed = False

force_install = False #@param{type:"boolean"}
if not everything_installed or force_install:
    install_everything()
elif everything_installed:
    from pytti.Notebook import change_tqdm_color
    change_tqdm_color()

/Users/brendan/src/github.com/wizardhead/pytti-notebook
Cloning into 'pytti-core'...
fatal: remote error: 
  The unauthenticated git protocol on port 9418 is no longer supported.
Please see https://github.blog/2021-09-01-improving-git-protocol-security-github/ for more information.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.4/493.4 KB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 6.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 KB 6.7 MB/s eta 0:00:0000:01
  Using cached pyDeprecate-0.3.1-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 9.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ModuleNotFoundError: No module named 'pytti'

# Step 2: Configure Experiment

Edit the parameters, or load saved parameters, then run the model.

* https://pytti-tools.github.io/pytti-book/SceneDSL.html
* https://pytti-tools.github.io/pytti-book/Settings.html

In [ ]:
#@title #2.1 Parameters:
#@markdown ---
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import change_tqdm_color, get_last_file
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()

import glob, json, random, re, math
try:
  from bunch import Bunch
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')

#these are used to make the defaults look pretty
model_default = None
random_seed = None
all  = math.inf
derive_from_init_aspect_ratio = -1

def define_parameters():
  locals_before = locals().copy()
  #@markdown ###Prompts:
  
  scenes = "deep space habitation ring made of glass | galactic nebula | wow! space is full of fractal creatures darting around everywhere like fireflies"#@param{type:"string"}
  scene_prefix = "astrophotography #pixelart | image credit nasa | space full of cybernetic neon:3_galactic nebula | isometric pixelart by Sachin Teng | "#@param{type:"string"}
  scene_suffix = "| satellite image:-1:-.95 | text:-1:-.95 | anime:-1:-.95 | watermark:-1:-.95 | backyard telescope:-1:-.95 | map:-1:-.95"#@param{type:"string"}
  interpolation_steps = 0#@param{type:"number"}
  steps_per_scene =  60100#@param{type:"raw"}
  #@markdown ---
  #@markdown ###Image Prompts:
  direct_image_prompts   = ""#@param{type:"string"}
  #@markdown ---
  #@markdown ###Initial image:
  init_image = ""#@param{type:"string"}
  direct_init_weight =  ""#@param{type:"string"}
  semantic_init_weight = ""#@param{type:"string"}
  #@markdown ---
  #@markdown ###Image:
  #@markdown Use `image_model` to select how the model will encode the image
  image_model = "Limited Palette" #@param ["VQGAN", "Limited Palette", "Unlimited Palette"]

  #@markdown image_model | description | strengths | weaknesses
  #@markdown --- | -- | -- | --
  #@markdown  VQGAN | classic VQGAN image | smooth images | limited datasets, slow, VRAM intesnsive 
  #@markdown  Limited Palette | pytti differentiable palette | fast,  VRAM scales with `palettes` | pixel images
  #@markdown  Unlimited Palette | simple RGB optimization | fast, VRAM efficient | pixel images
  
  #@markdown The output image resolution will be `width` $\times$ `pixel_size` by height $\times$ `pixel_size` pixels.
  #@markdown The easiest way to run out of VRAM is to select `image_model` VQGAN without reducing
  #@markdown `pixel_size` to $1$.

  #@markdown For `animation_mode: 3D` the minimum resoultion is about 450 by 400 pixels.
  width =  180#@param {type:"raw"}
  height =  112#@param {type:"raw"}
  pixel_size = 4#@param{type:"number"}
  smoothing_weight =  0.02#@param{type:"number"}
  #@markdown `VQGAN` specific settings:
  vqgan_model = "sflckr" #@param ["imagenet", "coco", "wikiart", "sflckr", "openimages"]
  #@markdown `Limited Palette` specific settings:
  random_initial_palette = False#@param{type:"boolean"}
  palette_size = 6#@param{type:"number"}
  palettes   = 9#@param{type:"number"}
  gamma = 1#@param{type:"number"}
  hdr_weight = 0.01#@param{type:"number"}
  palette_normalization_weight = 0.2#@param{type:"number"}
  show_palette = False #@param{type:"boolean"}
  target_palette = ""#@param{type:"string"}
  lock_palette = False #@param{type:"boolean"}
  #@markdown ---
  #@markdown ###Animation:
  animation_mode = "3D" #@param ["off","2D", "3D", "Video Source"]
  sampling_mode = "bicubic" #@param ["bilinear","nearest","bicubic"]
  infill_mode = "wrap" #@param ["mirror","wrap","black","smear"]
  pre_animation_steps =  100#@param{type:"number"}
  steps_per_frame =  50#@param{type:"number"}
  frames_per_second =  12#@param{type:"number"}
  #@markdown ---
  #@markdown ###Stabilization Weights:
  direct_stabilization_weight = ""#@param{type:"string"}
  semantic_stabilization_weight = ""#@param{type:"string"}
  depth_stabilization_weight = ""#@param{type:"string"}
  edge_stabilization_weight = ""#@param{type:"string"}
  #@markdown `flow_stabilization_weight` is used for `animation_mode: 3D` and `Video Source`
  flow_stabilization_weight = ""#@param{type:"string"}
  #@markdown ---
  #@markdown ###Video Tracking:
  #@markdown Only for `animation_mode: Video Source`.
  video_path = ""#@param{type:"string"}
  frame_stride = 1#@param{type:"number"}
  reencode_each_frame = True #@param{type:"boolean"}
  flow_long_term_samples = 1#@param{type:"number"}
  #@markdown ---
  #@markdown ###Image Motion:
  translate_x    = "-1700*sin(radians(1.5))" #@param{type:"string"}
  translate_y    = "0" #@param{type:"string"}
  #@markdown `..._3d` is only used in 3D mode.
  translate_z_3d = "(50+10*t)*sin(t/10*pi)**2" #@param{type:"string"}
  #@markdown `rotate_3d` *must* be a `[w,x,y,z]` rotation (unit) quaternion. Use `rotate_3d: [1,0,0,0]` for no rotation.
  #@markdown [Learn more about rotation quaternions here](https://eater.net/quaternions).
  rotate_3d      = "[cos(radians(1.5)), 0, -sin(radians(1.5))/sqrt(2), sin(radians(1.5))/sqrt(2)]"#@param{type:"string"}
  #@markdown `..._2d` is only used in 2D mode.
  rotate_2d      = "5" #@param{type:"string"}
  zoom_x_2d      = "0" #@param{type:"string"}
  zoom_y_2d      = "0" #@param{type:"string"}
  #@markdown  3D camera (only used in 3D mode):
  lock_camera   = True#@param{type:"boolean"}
  field_of_view = 60#@param{type:"number"}
  near_plane    = 1#@param{type:"number"}
  far_plane     = 10000#@param{type:"number"}

  #@markdown ---
  #@markdown ###Output:
  file_namespace = "default"#@param{type:"string"}
  if file_namespace == '':
    file_namespace = 'out'
  allow_overwrite = False#@param{type:"boolean"}
  base_name = file_namespace
  if not allow_overwrite and path_exists(f'images_out/{file_namespace}'):
    _, i = get_last_file(f'images_out/{file_namespace}', 
                         f'^(?P<pre>{re.escape(file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
    if i == 0:
      print(f"WARNING: file_namespace {file_namespace} already has images from run 0")
    elif i is not None:
      print(f"WARNING: file_namespace {file_namespace} already has images from runs 0 through {i}")
  elif glob.glob(f'images_out/{file_namespace}/{base_name}_*.png'):
    print(f"WARNING: file_namespace {file_namespace} has images which will be overwritten")
  try:
    del i
    del _
  except NameError:
    pass
  del base_name
  display_every = steps_per_frame #@param{type:"raw"}
  clear_every = 0 #@param{type:"raw"}
  display_scale = 1#@param{type:"number"}
  save_every = steps_per_frame #@param{type:"raw"}
  backups =  2**(flow_long_term_samples+1)+1#this is used for video transfer, so don't lower it if that's what you're doing#@param {type:"raw"}
  show_graphs = False #@param{type:"boolean"}
  approximate_vram_usage = False#@param{type:"boolean"}

  #@markdown ---
  #@markdown ###Model:
  #@markdown Quality settings from Dribnet's CLIPIT (https://github.com/dribnet/clipit).
  #@markdown Selecting too many will use up all your VRAM and slow down the model.
  #@markdown I usually use ViTB32, ViTB16, and RN50 if I get a A100, otherwise I just use ViT32B.

  #@markdown quality | CLIP models
  #@markdown --- | --
  #@markdown  draft | ViTB32 
  #@markdown  normal | ViTB32, ViTB16 
  #@markdown  high | ViTB32, ViTB16, RN50
  #@markdown  best | ViTB32, ViTB16, RN50x4
  ViTB32 = True #@param{type:"boolean"}
  ViTB16 = False #@param{type:"boolean"}
  RN50 = False #@param{type:"boolean"}
  RN50x4 = False #@param{type:"boolean"}
  ViTL14  = False #@param{type:"boolean"}
  RN101 = False #@param{type:"boolean"}
  RN50x16 = False #@param{type:"boolean"}
  RN50x64 = False #@param{type:"boolean"}
  #@markdown the default learning rate is `0.1` for all the VQGAN models
  #@markdown except openimages, which is `0.15`. For the palette modes the
  #@markdown default is `0.02`. 
  learning_rate =  model_default#@param{type:"raw"}
  reset_lr_each_frame = True#@param{type:"boolean"}
  seed = random_seed #@param{type:"raw"}
  #@markdown **Cutouts**:

  #@markdown [Cutouts are how CLIP sees the image.](https://twitter.com/remi_durant/status/1460607677801897990)
  cutouts =  40#@param{type:"number"}
  cut_pow =  2#@param {type:"number"}
  cutout_border =  .25#@param {type:"number"}
  gradient_accumulation_steps = 1 #@param {type:"number"}
  #@markdown NOTE: prompt masks (`promt:weight_[mask.png]`) will not work right on '`wrap`' or '`mirror`' mode.
  border_mode = "clamp" #@param ["clamp","mirror","wrap","black","smear"]
  models_parent_dir = '.'
  
  if seed is None:
    seed = random.randint(-0x8000_0000_0000_0000, 0xffff_ffff_ffff_ffff)
  locals_after = locals().copy()
  for k in locals_before.keys():
    del locals_after[k]
  del locals_after['locals_before']
  return locals_after

params = Bunch(define_parameters())
print("SETTINGS:")
print(json.dumps(params))

In [ ]:
#@title 2.2 Load settings (optional)
#@markdown copy the `SETTINGS:` output from the **Parameters** cell (tripple click to select the whole
#@markdown line from `{'scenes'...` to `}`) and paste them in a note to save them for later.

#@markdown Paste them here in the future to load those settings again. Running this cell with blank settings won't do anything.
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import *
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()
  
import json, random
try:
  from bunch import Bunch
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')

settings = ""#@param{type:"string"}
#@markdown Check `random_seed` to overwrite the seed from the settings with a random one for some variation.
random_seed = False #@param{type:"boolean"}

if settings != '':
  params = load_settings(settings, random_seed)

In [ ]:
from pytti.workhorse import TB_LOGDIR 

%load_ext tensorboard

In [ ]:
%tensorboard --logdir $TB_LOGDIR 

It is common for users to experience issues starting their first run. In particular, you may see an error saying something like "Access Denied" and showing you some URL links. This is caused by the google drive link for one of the models getting "hugged to death". You can still access the model, but google won't let you do it programmatically. Please follow these steps to get around the issue:

1. Visit either of the two URLs you see in your browser to download the file `AdaBins_nyu.pt` locally
2. Create a new folder in colab named `pretrained` (check the left sidebar for a file browser)
3. Upload `AdaBins_nyu.pt` to the `pretrained` folder. You should be able to just drag-and-drop the file onto the folder.
4. Run the following code cell after the upload has completed to tell PyTTI where to find AdaBins

You should now be able to run image generation without issues.

In [ ]:
%%sh

ADABINS_SRC=./pretrained/AdaBins_nyu.pt
ADABINS_DIR=~/.cache/adabins
ADABINS_TGT=$ADABINS_DIR/AdaBins_nyu.pt

if [ -f "$ADABINS_SRC" ]; then
    mkdir -p $ADABINS_DIR/
    ln $ADABINS_SRC $ADABINS_TGT
fi

In [ ]:
#@title 2.3 Run it!
from pytti.workhorse import _main as render_frames
from omegaconf import OmegaConf
cfg = OmegaConf.create(dict(params))

# function wraps step 2.3 of the original p5 notebook
render_frames(cfg)

# Step 3: Render video
You can dowload from the notebook, but it's faster to download from your drive.

In [ ]:
#@title 3.1 Render video
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import change_tqdm_color
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()
  
from tqdm.notebook import tqdm
import numpy as np
from os.path import exists as path_exists
from subprocess import Popen, PIPE
from PIL import Image, ImageFile
from os.path import splitext as split_file
import glob
from pytti.Notebook import get_last_file

ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
  params
except NameError:
  raise RuntimeError("ERROR: no parameters. Please run parameters (step 2.1).")

if not path_exists(f"images_out/{params.file_namespace}"):
  if path_exists(f"/content/drive/MyDrive"):
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"ERROR: file_namespace: {params.file_namespace} does not exist.")
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: file_namespace: {params.file_namespace} does not exist.")

#@markdown The first run executed in `file_namespace` is number $0$, the second is number $1$, etc.

latest = -1
run_number = latest#@param{type:"raw"}
if run_number == -1:
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  run_number = i
base_name = params.file_namespace if run_number == 0 else (params.file_namespace+f"({run_number})")
tqdm.write(f'Generating video from {params.file_namespace}/{base_name}_*.png')

all_frames = glob.glob(f'images_out/{params.file_namespace}/{base_name}_*.png')
all_frames.sort(key = lambda s: int(split_file(s)[0].split('_')[-1]))
print(f'found {len(all_frames)} frames matching images_out/{params.file_namespace}/{base_name}_*.png')

start_frame = 0#@param{type:"number"}
all_frames = all_frames[start_frame:]

fps =  params.frames_per_second#@param{type:"raw"}

total_frames = len(all_frames)

if total_frames == 0:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError(f"ERROR: no frames to render in images_out/{params.file_namespace}")

frames = []

for filename in tqdm(all_frames):
  frames.append(Image.open(filename))

p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f"videos/{base_name}.mp4"], stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video complete.")

In [ ]:
#@title 3.1 Render video (concatenate all runs)
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import change_tqdm_color
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')
change_tqdm_color()
  
from tqdm.notebook import tqdm
import numpy as np
from os.path import exists as path_exists
from subprocess import Popen, PIPE
from PIL import Image, ImageFile
from os.path import splitext as split_file
import glob
from pytti.Notebook import get_last_file

ImageFile.LOAD_TRUNCATED_IMAGES = True

try:
  params
except NameError:
  raise RuntimeError("ERROR: no parameters. Please run parameters (step 2.1).")

if not path_exists(f"images_out/{params.file_namespace}"):
  if path_exists(f"/content/drive/MyDrive"):
    raise RuntimeError(f"ERROR: file_namespace: {params.file_namespace} does not exist.")
  else:
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: file_namespace: {params.file_namespace} does not exist.")

#@markdown The first run executed in `file_namespace` is number $0$, the second is number $1$, etc.

latest = -1
run_number = latest
if run_number == -1:
  _, i = get_last_file(f'images_out/{params.file_namespace}', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?_1\\.png)$')
  run_number = i

all_frames = []
for i in range(run_number+1):
  base_name = params.file_namespace if i == 0 else (params.file_namespace+f"({i})")
  frames = glob.glob(f'images_out/{params.file_namespace}/{base_name}_*.png')
  frames.sort(key = lambda s: int(split_file(s)[0].split('_')[-1]))
  all_frames.extend(frames)

start_frame = 0#@param{type:"number"}
all_frames = all_frames[start_frame:]

fps =  params.frames_per_second#@param{type:"raw"}

total_frames = len(all_frames)

if total_frames == 0:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError(f"ERROR: no frames to render in images_out/{params.file_namespace}")

frames = []

for filename in tqdm(all_frames):
  frames.append(Image.open(filename))

p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', f"videos/{base_name}.mp4"], stdin=PIPE)
for im in tqdm(frames):
  im.save(p.stdin, 'PNG')
p.stdin.close()

print("Encoding video...")
p.wait()
print("Video complete.")

In [ ]:
#@title 3.2 Download the last exported video
from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}

try:
  from pytti.Notebook import get_last_file
except ModuleNotFoundError:
  if drive_mounted:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('ERROR: please run setup (step 1.3).')
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1.3).')

try:
  params
except NameError:
  #THIS IS NOT AN ERROR. This is the code that would
  #make an error if something were wrong.
  raise RuntimeError("ERROR: please run parameters (step 2.1).")

from google.colab import files
try:
  base_name = params.file_namespace if run_number == 0 else (params.file_namespace+f"({run_number})")
  filename = f'{base_name}.mp4'
except NameError:
  filename, i = get_last_file(f'videos', 
                       f'^(?P<pre>{re.escape(params.file_namespace)}\\(?)(?P<index>\\d*)(?P<post>\\)?\\.mp4)$')

if path_exists(f'videos/{filename}'):
  files.download(f"videos/{filename}")
else:
  if path_exists(f"/content/drive/MyDrive"):
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"ERROR: video videos/{filename} does not exist.")
  else:
    #THIS IS NOT AN ERROR. This is the code that would
    #make an error if something were wrong.
    raise RuntimeError(f"WARNING: Drive is not mounted.\nERROR: video videos/{filename} does not exist.")

# Batch Settings

Be Advised: google may penalize you for sustained colab GPU utilization, even if you are a PRO+ subscriber. Tread lightly with batch runs, you don't wanna end up in GPU jail.

FYI: the batch setting feature below may not work at present. We recommend using the CLI for batch jobs, see usage instructions at https://github.com/pytti-tools/pytti-core . The code below will probably be removed in the near future.

# Batch Setings
WARNING: If you use google colab (even with pro and pro+) GPUs for long enought google will throttle your account. Be careful with batch runs if you don't want to get kicked.

In [ ]:
#@title batch settings

# ngl... this probably doesn't work right now.

from os.path import exists as path_exists
if path_exists(gdrive_fpath):
  %cd {gdrive_fpath}
  drive_mounted = True
else:
  drive_mounted = False
try:
  from pytti.Notebook import change_tqdm_color, save_batch
except ModuleNotFoundError:
  if drive_mounted:
    raise RuntimeError('ERROR: please run setup (step 1).')
  else:
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1).')
change_tqdm_color()

try:
  import exrex, random, glob
except ModuleNotFoundError:
  if drive_mounted:
    raise RuntimeError('ERROR: please run setup (step 1).')
  else:
    raise RuntimeError('WARNING: drive is not mounted.\nERROR: please run setup (step 1).')
from numpy import arange
import itertools

def all_matches(s):
  return list(exrex.generate(s))

def dict_product(dictionary):
  return [dict(zip(dictionary, x)) for x in itertools.product(*dictionary.values())]

#these are used to make the defaults look pretty
model_default = None
random_seed = None

def define_parameters():
  locals_before = locals().copy()
  scenes = ["list","your","runs"] #@param{type:"raw"}
  scene_prefix = ["all "," permutations "," are run "] #@param{type:"raw"}
  scene_suffix = [" that", " makes", " 27" ] #@param{type:"raw"}
  interpolation_steps = [0] #@param{type:"raw"}
  steps_per_scene = [300] #@param{type:"raw"}
  direct_image_prompts = [""] #@param{type:"raw"}
  init_image = [""] #@param{type:"raw"}
  direct_init_weight = [""] #@param{type:"raw"}
  semantic_init_weight = [""] #@param{type:"raw"}
  image_model = ["Limited Palette"] #@param{type:"raw"}
  width = [180] #@param{type:"raw"}
  height = [112] #@param{type:"raw"}
  pixel_size = [4] #@param{type:"raw"}
  smoothing_weight = [0.05] #@param{type:"raw"}
  vqgan_model = ["sflckr"] #@param{type:"raw"}
  random_initial_palette = [False] #@param{type:"raw"}
  palette_size = [9] #@param{type:"raw"}
  palettes = [8] #@param{type:"raw"}
  gamma = [1] #@param{type:"raw"}
  hdr_weight = [1.0] #@param{type:"raw"}
  palette_normalization_weight = [1.0] #@param{type:"raw"}
  show_palette = [False] #@param{type:"raw"}
  target_palette = [""] #@param{type:"raw"}
  lock_palette = [False] #@param{type:"raw"}
  animation_mode = ["off"] #@param{type:"raw"}
  sampling_mode = ["bicubic"] #@param{type:"raw"}
  infill_mode = ["wrap"] #@param{type:"raw"}
  pre_animation_steps = [100] #@param{type:"raw"}
  steps_per_frame = [50] #@param{type:"raw"}
  frames_per_second = [12] #@param{type:"raw"}
  direct_stabilization_weight = [""] #@param{type:"raw"}
  semantic_stabilization_weight = [""] #@param{type:"raw"}
  depth_stabilization_weight = [""] #@param{type:"raw"}
  edge_stabilization_weight = [""] #@param{type:"raw"}
  flow_stabilization_weight = [""] #@param{type:"raw"}
  video_path = [""] #@param{type:"raw"}
  frame_stride = [1] #@param{type:"raw"}
  reencode_each_frame = [True] #@param{type:"raw"}
  flow_long_term_samples = [0] #@param{type:"raw"}
  translate_x = ["0"] #@param{type:"raw"}
  translate_y = ["0"] #@param{type:"raw"}
  translate_z_3d = ["0"] #@param{type:"raw"}
  rotate_3d = ["[1,0,0,0]"] #@param{type:"raw"}
  rotate_2d = ["0"] #@param{type:"raw"}
  zoom_x_2d = ["0"] #@param{type:"raw"}
  zoom_y_2d = ["0"] #@param{type:"raw"}
  lock_camera = [True] #@param{type:"raw"}
  field_of_view = [60] #@param{type:"raw"}
  near_plane = [1] #@param{type:"raw"}
  far_plane = [10000] #@param{type:"raw"}
  file_namespace = ["Basic Batch"] #@param{type:"raw"}
  allow_overwrite = [False]
  display_every = [50] #@param{type:"raw"}
  clear_every = [0] #@param{type:"raw"}
  display_scale = [1] #@param{type:"raw"}
  save_every = [50] #@param{type:"raw"}
  backups = [2] #@param{type:"raw"}
  show_graphs = [False] #@param{type:"raw"}
  approximate_vram_usage = [False] #@param{type:"raw"}
  ViTB32 = [True] #@param{type:"raw"}
  ViTB16 = [False] #@param{type:"raw"}
  RN50 = [False] #@param{type:"raw"}
  RN50x4 = [False] #@param{type:"raw"}
  learning_rate = [None] #@param{type:"raw"}
  reset_lr_each_frame = [True] #@param{type:"raw"}
  seed = [None] #@param{type:"raw"}
  cutouts = [40] #@param{type:"raw"}
  cut_pow = [2] #@param{type:"raw"}
  cutout_border = [0.25] #@param{type:"raw"}
  border_mode = ["clamp"] #@param{type:"raw"}
  locals_after = locals().copy()
  for k in locals_before.keys():
    del locals_after[k]
  del locals_after['locals_before']
  return locals_after

param_dict = define_parameters()
batch_list = dict_product(param_dict)
namespace = batch_list[0]['file_namespace']
if glob.glob(f'images_out/{namespace}/*.png'):
  print(f"WARNING: images_out/{namespace} contains images. Batch indicies may not match filenames unless restoring.")

In [ ]:
# @title Licensed under the MIT License
# Copyleft (c) 2021 Henry Rachootin
# Copyright (c) 2022 David Marx

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.